## Markov Decision Process

- 给定一个马尔可夫过程, 可以从某个状态出发, 根据它的状态转移矩阵生成一个状态序列(episode), 这个步骤也被叫做采样(sampling)

### 马尔可夫奖励过程
马尔可夫奖励过程（Markov Reward Process, MRP）由元组 $\langle S, P, R, \gamma \rangle$ 组成：
- $S$ 是有限状态集
- $P$ 是状态转移概率矩阵，$P(s'|s) = \mathbb{P}[S_{t+1}=s'|S_t=s]$
- $R$ 是奖励函数，$R(s) = \mathbb{E}[R_{t+1}|S_t=s]$
- $\gamma$ 是折扣因子，$\gamma \in [0,1]$

**回报（Return）**定义：
$$ G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum_{k=0}^\infty \gamma^k R_{t+k+1} $$

**状态价值函数（State Value Function）**：
$$ V(s) = \mathbb{E}[G_t|S_t=s] $$

**贝尔曼方程**：
$$ V(s) = R(s) + \gamma \sum_{s' \in S} P(s'|s)V(s') $$

In [1]:
import numpy as np
np.random.seed(0)
# 定义状态转移概率矩阵P
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)

rewards = [-1, -2, -2, 10, 1, 0]  # 定义奖励函数
gamma = 0.5  # 定义折扣因子


# 给定一条序列,计算从某个索引（起始状态）开始到序列最后（终止状态）得到的回报
def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G = gamma * G + rewards[chain[i] - 1]
    return G


# 一个状态序列,s1-s2-s3-s6
chain = [1, 2, 3, 6]
start_index = 0
G = compute_return(start_index, chain, gamma)
print("根据本序列计算得到回报为：%s。" % G)

根据本序列计算得到回报为：-2.5。
